Statistical Analysis of Results
===============================
This will be an examnination of results. Just statistical analysis nothing else.

# Table of Contents
1. [Preliminaries](#1.-preliminaries)
2. [Kling-Gupta Efficiency](#2.-kling-gupta-efficiency)
   * [isoNet](#isoNet-KGE)
   * [isoP](#isoP-KGE)
3. [Root Mean Square Error](#3.-root-mean-square-error)
   * [isoNet](#isoNet-RMSE)
   * [isoP](#isoP-RMSE)
4. [Combining Results](#4.-combining-results)

# 1. Preliminaries
This is the setup for the rest of the analysis.

In [2]:
# Library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Load in the isotope data that it was trained on, and extract the lat lon of the individual stations
isotope_data = pd.read_csv('Isoscape_Data.csv')
isotope_data.drop_duplicates(subset=['Station'], inplace=True)
isotope_data.reset_index()

station_coord = isotope_data[['Station', 'Lat', 'Long']]
station_coord.set_index('Station', inplace=True)
station_coord = station_coord.to_dict(orient='index')
station_coord

{'RES': {'Lat': 74.43, 'Long': 94.59},
 'HAL': {'Lat': 68.47, 'Long': 81.15},
 'CAM': {'Lat': 69.6, 'Long': 105.8},
 'OTT': {'Lat': 45.32, 'Long': 75.67},
 'BAB': {'Lat': 47.98, 'Long': 55.82},
 'SNA': {'Lat': 63.52, 'Long': 116.0},
 'SKT': {'Lat': 52.1, 'Long': 106.43},
 'ELA': {'Lat': 49.67, 'Long': 93.72},
 'SAT': {'Lat': 48.78, 'Long': 123.13},
 'HAB': {'Lat': 46.29, 'Long': 64.15},
 'CPA': {'Lat': 49.82, 'Long': 74.97},
 'BON': {'Lat': 49.38, 'Long': 82.12},
 'EGB': {'Lat': 44.23, 'Long': 79.77},
 'GOB': {'Lat': 53.32, 'Long': 60.42},
 'EST': {'Lat': 51.67, 'Long': 110.2}}

In [49]:
# Load in results into a pandas dataframe
results = pd.read_csv('results_test.csv')

# Convert the day of year column and year column into a datetime object
results['date'] = pd.to_datetime(results['Year'].astype(str) + '-' + results['Day'].astype(str), format='%Y-%j')

results = results.drop(columns=['Year', 'Day'])

# Create new column for the station name and fill it with the station name based off the lat lon and the station_coord dataframe
for stat in station_coord:
    results.loc[results['Lat'] == station_coord[stat]['Lat'], 'Station'] = stat

# Change the date to start on the first of each month, instead of the second
results.date = results.date - pd.Timedelta('1D')

results.head()

,Lat,Long,Alt,Precipitation (kg/m^2/s),Temperature (K),Predictions,Actual,date,Station
0,47.98,55.82,190.0,0.0,271.142517,-12.983921,-16.59,2003-12-01,BAB
1,49.38,82.12,245.0,0.0,265.255096,-20.723955,-20.07,2003-12-01,BON
2,46.29,64.15,45.0,0.0,270.236603,-14.407454,-8.42,2003-12-01,HAB
3,45.32,75.67,114.0,0.0,269.147888,-13.856897,-13.85,2003-12-01,OTT
4,48.78,123.13,178.0,0.0,252.437027,-16.355522,-12.78,2003-12-01,SAT


# 2. Kling-Gupta Efficiency
In this I will be breaking down the Kling-Gupta Efficiency (KGE) into its components and then analyzing the results. Finally also storing them in a dataframe for later examination.

## isoNet KGE

In [50]:
# Create a new dataframe consisting of the mean and standard deviation of the isotope values for each station, and for the Prediction column and Actual column
station_stats = results["Predictions"].groupby(results["Station"]).agg(['mean', 'std'])
station_stats.rename(columns={'mean': 'Pred_Mean', 'std': 'Pred_Std'}, inplace=True)
station_stats['Actual_Mean'] = results["Actual"].groupby(results["Station"]).mean()
station_stats['Actual_Std'] = results["Actual"].groupby(results["Station"]).std()

# Create a new column consisting of the pearson correlation coefficient between the Predictions and the Actual values for each station
station_stats['Corr'] = results["Predictions"].groupby(results["Station"]).corr(results["Actual"])
station_stats

,Pred_Mean,Pred_Std,Actual_Mean,Actual_Std,Corr
Station,,,,,
BAB,-11.221481,3.170715,-9.807308,3.164775,0.648743
BON,-17.060243,4.937385,-15.680769,6.310736,0.831113
CPA,-16.610241,4.562381,-15.816750,5.421716,0.832348
ELA,-17.083738,5.581105,-14.960750,5.598884,0.879654
GOB,-16.503662,4.576316,-15.465256,3.871378,0.798785
HAB,-12.324240,2.995911,-11.179878,4.096467,0.662307
OTT,-12.213251,3.263574,-10.765862,4.100087,0.729063
SAT,-11.620949,2.789616,-9.844875,2.075112,0.728224
SNA,-21.940439,3.767560,-22.307219,4.420089,0.021694


In [51]:
# Create new dataframe just for the KGE components and values
kge = pd.DataFrame(columns=['alpha', 'beta', 'r', 'kge'], index=station_stats.index)

# Fill in the KGE dataframe with the alpha value (variablility ratio)
kge['alpha'] = station_stats['Pred_Std'] / station_stats['Actual_Std']

# Fill in the KGE dataframe with the beta value (bias ratio)
kge['beta'] = (station_stats['Pred_Mean'] - station_stats['Actual_Mean']) / station_stats['Actual_Mean']

# Fill in the KGE dataframe with the r value (correlation coefficient)
kge['r'] = station_stats['Corr']

# Fill in the KGE dataframe with the KGE value
kge['kge'] = 1 - np.sqrt((kge['alpha'] - 1)**2 + (kge['beta'])**2 + (1 - kge['r'])**2)

kge

,alpha,beta,r,kge
Station,,,,
BAB,1.001877,0.144196,0.648743,0.620293
BON,0.782379,0.087972,0.831113,0.710827
CPA,0.841501,0.050168,0.832348,0.763895
ELA,0.996825,0.141904,0.879654,0.813909
GOB,1.182090,0.067144,0.798785,0.720443
HAB,0.731340,0.102359,0.662307,0.556500
OTT,0.795977,0.134442,0.729063,0.635162
SAT,1.344320,0.180406,0.728224,0.525695
SNA,0.852372,-0.016442,0.021694,0.010481


## isoP KGE
Performing the same calculations but this time just for the isoP results

In [52]:
# Load in Data from isoP
isoP = pd.read_csv('isoP_Output.csv')

# Change longitudes to be positive
isoP['Lon'] = isoP['Lon'] * -1

# Add in the station name to the isoP dataframe with the station_coord dataframe
for stat in station_coord:
    isoP.loc[isoP['Lat'] == station_coord[stat]['Lat'], 'Station'] = stat

# Combine Month and Year into a single column and convert to datetime object
isoP['date'] = pd.to_datetime(isoP['Year'].astype(str) + '-' + isoP['Month'].astype(str), format='%Y-%m')
isoP.drop(columns=['Year', 'Month'], inplace=True)

# Reorder the columns
isoP = isoP[['date', 'Station', 'Lat', 'Lon', 'isoP']]

# Remove any rows in isoP that do not have a corresponding row in the results dataframe
isoP = isoP[isoP['date'].isin(results['date'])]

# Merge the isoP dataframe with the results dataframe on the date and station columns, keeping only the actual and isoP columns
isoP = isoP.merge(results, on=['date', 'Station'], how='inner')

isoP.drop(columns=['Lat_y', 'Long', 'Precipitation (kg/m^2/s)', 'Temperature (K)', 'Predictions'], inplace=True)
isoP.rename(columns={'Lat_x': 'Lat'}, inplace=True)
isoP

,date,Station,Lat,Lon,isoP,Alt,Actual
0,2004-01-01,OTT,45.32,75.67,-20.112738,114.0,-18.920
1,2004-02-01,OTT,45.32,75.67,-15.957769,114.0,-15.080
2,2004-03-01,OTT,45.32,75.67,-15.134004,114.0,-10.890
3,2004-04-01,OTT,45.32,75.67,-12.415049,114.0,-10.990
4,2004-05-01,OTT,45.32,75.67,-9.781096,114.0,-7.080
...,...,...,...,...,...,...,...
343,2006-11-01,GOB,53.32,60.42,-15.551108,46.0,-13.550
344,2006-12-01,GOB,53.32,60.42,-18.446399,46.0,-22.410
345,2007-01-01,GOB,53.32,60.42,-19.344636,46.0,-23.070
346,2007-02-01,GOB,53.32,60.42,-19.597808,46.0,-20.530


In [53]:
# Now we can perform the same analysis as we did with the isoNet data
isoP_stats = isoP["isoP"].groupby(isoP["Station"]).agg(['mean', 'std'])
isoP_stats.rename(columns={'mean': 'isoP_Mean', 'std': 'isoP_Std'}, inplace=True)
isoP_stats['Actual_Mean'] = isoP["Actual"].groupby(isoP["Station"]).mean()
isoP_stats['Actual_Std'] = isoP["Actual"].groupby(isoP["Station"]).std()
isoP_stats['Corr'] = isoP["isoP"].groupby(isoP["Station"]).corr(isoP["Actual"])

isoP_stats

,isoP_Mean,isoP_Std,Actual_Mean,Actual_Std,Corr
Station,,,,,
BAB,-9.492269,2.532441,-9.628816,3.001733,0.598408
BON,-15.477240,5.197228,-15.565263,6.353531,0.835785
CPA,-15.619627,4.416323,-15.722564,5.459338,0.912512
ELA,-16.149326,5.225621,-14.960750,5.598884,0.802588
GOB,-15.450212,3.688603,-15.465256,3.871378,0.851669
HAB,-10.316499,2.967529,-11.248875,4.124456,0.693154
OTT,-11.081726,3.678253,-10.701609,4.118469,0.833848
SAT,-8.835244,2.683577,-9.769615,2.046188,0.786670
SNA,-21.985543,4.255335,-22.307219,4.420089,-0.341698


In [54]:
# Now the KGE values
isoP_kge = pd.DataFrame(columns=['alpha', 'beta', 'r', 'kge'], index=isoP_stats.index)

# Fill in the KGE dataframe with the alpha value (variablility ratio)
isoP_kge['alpha'] = isoP_stats['isoP_Std'] / isoP_stats['Actual_Std']

# Fill in the KGE dataframe with the beta value (bias ratio)
isoP_kge['beta'] = (isoP_stats['isoP_Mean'] - isoP_stats['Actual_Mean']) / isoP_stats['Actual_Mean']

# Fill in the KGE dataframe with the r value (correlation coefficient)
isoP_kge['r'] = isoP_stats['Corr']

# Fill in the KGE dataframe with the KGE value
isoP_kge['kge'] = 1 - np.sqrt((isoP_kge['alpha'] - 1)**2 + (isoP_kge['beta'])**2 + (1 - isoP_kge['r'])**2)

isoP_kge

,alpha,beta,r,kge
Station,,,,
BAB,0.843660,-0.014181,0.598408,0.568816
BON,0.818006,-0.005655,0.835785,0.754806
CPA,0.808948,-0.006547,0.912512,0.789768
ELA,0.933333,0.079446,0.802588,0.777003
GOB,0.952788,-0.000973,0.851669,0.844334
HAB,0.719496,-0.082886,0.693154,0.576081
OTT,0.893112,0.035520,0.833848,0.799268
SAT,1.311500,-0.095641,0.786670,0.610527
SNA,0.962726,-0.014420,-0.341698,-0.342293


In [55]:
# Combine the isoNet and isoP KGE dataframes into a single dataframe
kge = pd.merge(isoP_kge, kge, left_index=True, right_index=True, suffixes=('_isoP', '_isoNet'))
kge = kge[['kge_isoP', 'kge_isoNet']]
kge

,kge_isoP,kge_isoNet
Station,,
BAB,0.568816,0.620293
BON,0.754806,0.710827
CPA,0.789768,0.763895
ELA,0.777003,0.813909
GOB,0.844334,0.720443
HAB,0.576081,0.556500
OTT,0.799268,0.635162
SAT,0.610527,0.525695
SNA,-0.342293,0.010481


# 3. Root Mean Square Error
In this I will be calculating the Root Mean Square Error (RMSE) 

## isoNet RMSE
Just for the current isoNet results

In [56]:
# In station_stats, calculate the RMSE for each station
station_stats['RMSE'] = np.sqrt(((results['Predictions'] - results['Actual']) ** 2).groupby(results['Station']).mean())
station_stats

,Pred_Mean,Pred_Std,Actual_Mean,Actual_Std,Corr,RMSE
Station,,,,,,
BAB,-11.221481,3.170715,-9.807308,3.164775,0.648743,2.978021
BON,-17.060243,4.937385,-15.680769,6.310736,0.831113,3.741034
CPA,-16.610241,4.562381,-15.816750,5.421716,0.832348,3.071857
ELA,-17.083738,5.581105,-14.960750,5.598884,0.879654,3.440999
GOB,-16.503662,4.576316,-15.465256,3.871378,0.798785,2.917082
HAB,-12.324240,2.995911,-11.179878,4.096467,0.662307,3.252363
OTT,-12.213251,3.263574,-10.765862,4.100087,0.729063,3.143756
SAT,-11.620949,2.789616,-9.844875,2.075112,0.728224,2.592301
SNA,-21.940439,3.767560,-22.307219,4.420089,0.021694,5.649878


## isoP RMSE
Now for the isoP results

In [58]:
isoP_stats['RMSE'] = np.sqrt(((isoP['isoP'] - isoP['Actual']) ** 2).groupby(isoP['Station']).mean())
isoP_stats

,isoP_Mean,isoP_Std,Actual_Mean,Actual_Std,Corr,RMSE
Station,,,,,,
BAB,-9.492269,2.532441,-9.628816,3.001733,0.598408,2.485556
BON,-15.477240,5.197228,-15.565263,6.353531,0.835785,3.445169
CPA,-15.619627,4.416323,-15.722564,5.459338,0.912512,2.276202
ELA,-16.149326,5.225621,-14.960750,5.598884,0.802588,3.579302
GOB,-15.450212,3.688603,-15.465256,3.871378,0.851669,2.039725
HAB,-10.316499,2.967529,-11.248875,4.124456,0.693154,3.081853
OTT,-11.081726,3.678253,-10.701609,4.118469,0.833848,2.294202
SAT,-8.835244,2.683577,-9.769615,2.046188,0.786670,1.884586
SNA,-21.985543,4.255335,-22.307219,4.420089,-0.341698,6.980842


# 4. Combining Results

In [59]:
# Create a dataframe for the RMSE values and the KGE values
final_stats = pd.DataFrame(columns=['RMSE_isoNet', 'RMSE_isoP', 'KGE_isoNet', 'KGE_isoP'], index=station_stats.index)

# Fill in the final_stats dataframe with the RMSE and KGE values
final_stats['RMSE_isoNet'] = station_stats['RMSE']
final_stats['RMSE_isoP'] = isoP_stats['RMSE']
final_stats['KGE_isoNet'] = kge['kge_isoNet']
final_stats['KGE_isoP'] = kge['kge_isoP']

final_stats

,RMSE_isoNet,RMSE_isoP,KGE_isoNet,KGE_isoP
Station,,,,
BAB,2.978021,2.485556,0.620293,0.568816
BON,3.741034,3.445169,0.710827,0.754806
CPA,3.071857,2.276202,0.763895,0.789768
ELA,3.440999,3.579302,0.813909,0.777003
GOB,2.917082,2.039725,0.720443,0.844334
HAB,3.252363,3.081853,0.556500,0.576081
OTT,3.143756,2.294202,0.635162,0.799268
SAT,2.592301,1.884586,0.525695,0.610527
SNA,5.649878,6.980842,0.010481,-0.342293


In [ ]:
# Export the final_stats dataframe to a csv file
final_stats.to_csv('results_stats.csv')